In [2]:
import os


In [3]:
%pwd

'c:\\MlOps\\DS\\Text-Summarization\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\MlOps\\DS\\Text-Summarization'

In [6]:
from datasets import load_dataset

# Load your dataset object (it won’t re‑download if it’s already cached)
ds = load_dataset("knkarthick/samsum")

# Delete all cache files for this dataset
deleted = ds.cleanup_cache_files()
print("Deleted cache files:", deleted)

C:\Users\User\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\anaconda3\envs\testS\Lib\importlib\__init__.py:90: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  return _bootstrap._gcd_import(name[level:], package, level)


Deleted cache files: {'train': 0, 'validation': 0, 'test': 0}


In [35]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [34]:
from pathlib import Path
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [9]:
from textSummarizer.constants import CONFIG_FILE_PATH
from pathlib import Path

print("CONFIG_FILE_PATH:", CONFIG_FILE_PATH)
print("Resolved:", CONFIG_FILE_PATH.resolve())
print("Exists:", CONFIG_FILE_PATH.exists())

with open(CONFIG_FILE_PATH, "r", encoding="utf-8") as f:
    content = f.read()
    print("\n---- File Content ----")
    print(content)
    print("----------------------")


CONFIG_FILE_PATH: config\config.yaml
Resolved: C:\MlOps\DS\Text-Summarization\config\config.yaml
Exists: True

---- File Content ----
artifacts_root: artifacts

data_ingestion:
  root_dir: artifacts/data_ingestion
  source_URL: https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
  local_data_file: artifacts/data_ingestion/data.zip
  unzip_dir: artifacts/data_ingestion

----------------------


In [36]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            PARAMS_FILE_PATH = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [37]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [40]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [41]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-19 10:13:31,814: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-06-19 10:13:31,816: INFO: common: YAML file: params.yaml loaded successfully]
[2025-06-19 10:13:31,819: INFO: common: created directory at: artifacts]
[2025-06-19 10:13:31,822: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-19 10:13:34,291: INFO: 915194583: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A322:201DEF:387353:4240DD:6853C6AC
Accept-Ranges: bytes
Date: Thu, 19 Jun 2025 08:13:32 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf8230021-FRA
X-Ca